In [1]:
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import string
from scipy.spatial.distance import pdist, squareform
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

## Corpus: list of documents

```
    ['This roller coaster is too rickety.', 
    'Roller coasters give me hives.', 
    'That bee hive is approaching at an alarming speed.']```

## Term frequency
$$TF_{word,document} = \frac{\#\_of\_times\_word\_appears\_in\_document}{total\_\#\_of\_words\_in\_document}$$

## Inverse document frequency
$$ IDF_{word} = \log\left(\frac{total\_\#\_of\_documents}{\#\_of\_documents\_containing\_word}\right) $$

In [2]:
corpus = ['This roller coaster is too, too rickety.', 
    'Roller coasters give me hives.', 
    'That bee hive is approaching at an alarming speed.']

In [3]:
tokenized_doc = [doc.lower().split() for doc in corpus]

In [4]:
tokenized_doc = [word_tokenize(doc.lower()) for doc in corpus]

In [5]:
def our_tokenizer(doc):
    doc = word_tokenize(doc.lower())
    return [tok for tok in doc if tok not in string.punctuation]

In [6]:
tokenized_docs = [our_tokenizer(doc) for doc in corpus]

In [7]:
tokenized_docs

[['this', 'roller', 'coaster', 'is', 'too', 'too', 'rickety'],
 ['roller', 'coasters', 'give', 'me', 'hives'],
 ['that', 'bee', 'hive', 'is', 'approaching', 'at', 'an', 'alarming', 'speed']]

In [8]:
vocab = set()

In [9]:
for doc in tokenized_docs:
    vocab.update(doc)

In [10]:
tokenized_docs

[['this', 'roller', 'coaster', 'is', 'too', 'too', 'rickety'],
 ['roller', 'coasters', 'give', 'me', 'hives'],
 ['that', 'bee', 'hive', 'is', 'approaching', 'at', 'an', 'alarming', 'speed']]

In [11]:
vocab = sorted(list(vocab))
vocab

['alarming',
 'an',
 'approaching',
 'at',
 'bee',
 'coaster',
 'coasters',
 'give',
 'hive',
 'hives',
 'is',
 'me',
 'rickety',
 'roller',
 'speed',
 'that',
 'this',
 'too']

In [12]:
tokenized_docs[0]

['this', 'roller', 'coaster', 'is', 'too', 'too', 'rickety']

In [13]:
Counter(tokenized_docs[0])

Counter({'coaster': 1,
         'is': 1,
         'rickety': 1,
         'roller': 1,
         'this': 1,
         'too': 2})

In [14]:
doc_vec = np.zeros(len(vocab))

In [15]:
doc_vec

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.])

In [16]:
for word, count in Counter(tokenized_docs[0]).iteritems():
    ind = vocab.index(word)
    doc_vec[ind] = count

In [17]:
doc_vec

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
        1.,  0.,  0.,  1.,  2.])

sklearn

In [18]:
vect = TfidfVectorizer(stop_words='english')
vector_matrix = vect.fit(corpus)

In [19]:
vect.get_feature_names()

[u'alarming',
 u'approaching',
 u'bee',
 u'coaster',
 u'coasters',
 u'hive',
 u'hives',
 u'rickety',
 u'roller',
 u'speed']

In [20]:
[word_tokenize(doc) for doc in corpus]

[['This', 'roller', 'coaster', 'is', 'too', ',', 'too', 'rickety', '.'],
 ['Roller', 'coasters', 'give', 'me', 'hives', '.'],
 ['That',
  'bee',
  'hive',
  'is',
  'approaching',
  'at',
  'an',
  'alarming',
  'speed',
  '.']]

In [21]:
def my_tokenizer(doc, lemmatizer=WordNetLemmatizer(), stopwords=None):
    tokens = word_tokenize(doc)
    tokens = [t.lower() for t in tokens if t not in string.punctuation]
    if lemmatizer:
        tokens = [lemmatizer.lemmatize(t) for t in tokens]
    if stopwords:
        tokens = [t for t in tokens if t not in stopwords]    
    return tokens

In [22]:
[my_tokenizer(doc, lemmatizer=None) for doc in corpus]

[['this', 'roller', 'coaster', 'is', 'too', 'too', 'rickety'],
 ['roller', 'coasters', 'give', 'me', 'hives'],
 ['that', 'bee', 'hive', 'is', 'approaching', 'at', 'an', 'alarming', 'speed']]

In [23]:
[my_tokenizer(doc) for doc in corpus]

[['this', 'roller', 'coaster', 'is', 'too', 'too', 'rickety'],
 ['roller', u'coaster', 'give', 'me', u'hive'],
 ['that', 'bee', 'hive', 'is', 'approaching', 'at', 'an', 'alarming', 'speed']]

In [24]:
vect = TfidfVectorizer(stop_words='english', tokenizer=my_tokenizer)
vector_matrix = vect.fit_transform(corpus)

In [25]:
print vect.get_feature_names()
print '---'
print vect.vocabulary_

[u'alarming', u'approaching', u'bee', u'coaster', u'hive', u'rickety', u'roller', u'speed']
---
{u'coaster': 3, u'alarming': 0, u'rickety': 5, u'hive': 4, u'bee': 2, u'speed': 7, u'roller': 6, u'approaching': 1}


In [26]:
vector_matrix

<3x8 sparse matrix of type '<type 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [27]:
print vector_matrix

  (0, 5)	0.680918560399
  (0, 3)	0.517856116168
  (0, 6)	0.517856116168
  (1, 4)	0.57735026919
  (1, 3)	0.57735026919
  (1, 6)	0.57735026919
  (2, 7)	0.467350981811
  (2, 0)	0.467350981811
  (2, 1)	0.467350981811
  (2, 2)	0.467350981811
  (2, 4)	0.35543246785


In [28]:
print vector_matrix.toarray()

[[ 0.          0.          0.          0.51785612  0.          0.68091856
   0.51785612  0.        ]
 [ 0.          0.          0.          0.57735027  0.57735027  0.
   0.57735027  0.        ]
 [ 0.46735098  0.46735098  0.46735098  0.          0.35543247  0.          0.
   0.46735098]]


In [29]:
vector_matrix.toarray()[1]

array([ 0.        ,  0.        ,  0.        ,  0.57735027,  0.57735027,
        0.        ,  0.57735027,  0.        ])

In [30]:
squareform(pdist(vector_matrix.toarray(), metric='cosine'))

array([[ 0.        ,  0.40203126,  1.        ],
       [ 0.40203126,  0.        ,  0.79479097],
       [ 1.        ,  0.79479097,  0.        ]])

# TF-IDF example: a villanelle
"The highly structured villanelle is a nineteen-line poem with two repeating rhymes and two refrains. The form is made up of five tercets followed by a quatrain. The first and third lines of the opening tercet are repeated alternately in the last lines of the succeeding stanzas; then in the final stanza, the refrain serves as the poem’s two concluding lines. Using capitals for the refrains and lowercase letters for the rhymes, the form could be expressed as: A1 b A2 / a b A1 / a b A2 / a b A1 / a b A2 / a b A1 A2."

In [31]:
raw_poem = '''Do not go gentle into that good night,
Old age should burn and rave at close of day;
Rage, rage against the dying of the light.

Though wise men at their end know dark is right,
Because their words had forked no lightning they
Do not go gentle into that good night.

Good men, the last wave by, crying how bright
Their frail deeds might have danced in a green bay,
Rage, rage against the dying of the light.

Wild men who caught and sang the sun in flight,
And learn, too late, they grieved it on its way,
Do not go gentle into that good night.

Grave men, near death, who see with blinding sight
Blind eyes could blaze like meteors and be gay,
Rage, rage against the dying of the light.

And you, my father, there on the sad height,
Curse, bless, me now with your fierce tears, I pray.
Do not go gentle into that good night.
Rage, rage against the dying of the light.'''

### Let's treat each line as a document and the poem as our corpus, then vectorize this baby into a matrix that would make Dylan Thomas proud (sorry).

In [32]:
lines = raw_poem.split('\n')
lines

['Do not go gentle into that good night,',
 'Old age should burn and rave at close of day;',
 'Rage, rage against the dying of the light.',
 '',
 'Though wise men at their end know dark is right,',
 'Because their words had forked no lightning they',
 'Do not go gentle into that good night.',
 '',
 'Good men, the last wave by, crying how bright',
 'Their frail deeds might have danced in a green bay,',
 'Rage, rage against the dying of the light.',
 '',
 'Wild men who caught and sang the sun in flight,',
 'And learn, too late, they grieved it on its way,',
 'Do not go gentle into that good night.',
 '',
 'Grave men, near death, who see with blinding sight',
 'Blind eyes could blaze like meteors and be gay,',
 'Rage, rage against the dying of the light.',
 '',
 'And you, my father, there on the sad height,',
 'Curse, bless, me now with your fierce tears, I pray.',
 'Do not go gentle into that good night.',
 'Rage, rage against the dying of the light.']

In [33]:
#removing empty lines
lines = [l for l in lines if l]
lines

['Do not go gentle into that good night,',
 'Old age should burn and rave at close of day;',
 'Rage, rage against the dying of the light.',
 'Though wise men at their end know dark is right,',
 'Because their words had forked no lightning they',
 'Do not go gentle into that good night.',
 'Good men, the last wave by, crying how bright',
 'Their frail deeds might have danced in a green bay,',
 'Rage, rage against the dying of the light.',
 'Wild men who caught and sang the sun in flight,',
 'And learn, too late, they grieved it on its way,',
 'Do not go gentle into that good night.',
 'Grave men, near death, who see with blinding sight',
 'Blind eyes could blaze like meteors and be gay,',
 'Rage, rage against the dying of the light.',
 'And you, my father, there on the sad height,',
 'Curse, bless, me now with your fierce tears, I pray.',
 'Do not go gentle into that good night.',
 'Rage, rage against the dying of the light.']

In [34]:
stops = nltk.corpus.stopwords.words('english')
stops

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all

In [35]:
print my_tokenizer(lines[7], lemmatizer=None)

['their', 'frail', 'deeds', 'might', 'have', 'danced', 'in', 'a', 'green', 'bay']


In [36]:
print my_tokenizer(lines[7])

['their', 'frail', u'deed', 'might', 'have', 'danced', 'in', 'a', 'green', 'bay']


In [37]:
print my_tokenizer(lines[7], lemmatizer=None, stopwords=set(stops))

['frail', 'deeds', 'might', 'danced', 'green', 'bay']


In [38]:
print my_tokenizer(lines[7], stopwords=set(stops))

['frail', u'deed', 'might', 'danced', 'green', 'bay']


In [39]:
def my_vectorizer(corpus, stopwords=None, lemmatizer=None, kind='count'):
    '''
    INPUT: list of strings (documents)
    OUTPUT: 2D numpy array (vector matrix), 
            1D numpy array (sorted vocabulary),
            dictionary (keys: vocab words, values: indices in sorted vocab)
    
    '''
    #Tokenize documents
    tokenized_docs = [my_tokenizer(doc, stopwords=stopwords, lemmatizer=lemmatizer) for doc in corpus]
    
    #Make an array of unique words in the corpus
    vocab_list = set()
    for doc in tokenized_docs:
        vocab_list.update(doc)
    vocab_list = np.array(sorted(list(vocab_list)))
    
    #Make a dictionary mapping vocab tokens (words) to indices in this list
    vocab_dict = dict()
    for i, token in enumerate(vocab_list):
        vocab_dict[token] = i
    
    #Vectorize each document!
    vector_matrix = np.zeros((len(corpus), len(vocab_list)))
    for i, doc in enumerate(tokenized_docs):
        counter = Counter(doc)
        for token, count in counter.iteritems():
            vector_matrix[i,vocab_dict[token]] = count
    
    # TF-IDF code
    if kind == 'tfidf':
        # number of documents
        N = len(corpus)
        
        # document frequency for each word in vocab
        df = np.array([sum(1 for doc in tokenized_docs if word in doc)\
                       for word in vocab_list])
        
        # idf
        idf_list = np.log(N/df)
        
        # vectorizin'
        vector_matrix = np.zeros((len(corpus), len(vocab_list)))
        for i, doc in enumerate(tokenized_docs):
            counter = Counter(doc)
            n = 1.*len(doc)
            for token, count in counter.iteritems():
                idf = idf_list[vocab_dict[token]]
                vector_matrix[i,vocab_dict[token]] = (count/n)*idf
                
    return vector_matrix, vocab_list, vocab_dict

In [40]:
vec_mat, voc_list, voc_dict = my_vectorizer(lines)

In [41]:
voc_list

array(['a', 'against', 'age', 'and', 'at', 'bay', 'be', 'because', 'blaze',
       'bless', 'blind', 'blinding', 'bright', 'burn', 'by', 'caught',
       'close', 'could', 'crying', 'curse', 'danced', 'dark', 'day',
       'death', 'deeds', 'do', 'dying', 'end', 'eyes', 'father', 'fierce',
       'flight', 'forked', 'frail', 'gay', 'gentle', 'go', 'good', 'grave',
       'green', 'grieved', 'had', 'have', 'height', 'how', 'i', 'in',
       'into', 'is', 'it', 'its', 'know', 'last', 'late', 'learn', 'light',
       'lightning', 'like', 'me', 'men', 'meteors', 'might', 'my', 'near',
       'night', 'no', 'not', 'now', 'of', 'old', 'on', 'pray', 'rage',
       'rave', 'right', 'sad', 'sang', 'see', 'should', 'sight', 'sun',
       'tears', 'that', 'the', 'their', 'there', 'they', 'though', 'too',
       'wave', 'way', 'who', 'wild', 'wise', 'with', 'words', 'you', 'your'], 
      dtype='|S9')

In [42]:
voc_dict

{'a': 0,
 'against': 1,
 'age': 2,
 'and': 3,
 'at': 4,
 'bay': 5,
 'be': 6,
 'because': 7,
 'blaze': 8,
 'bless': 9,
 'blind': 10,
 'blinding': 11,
 'bright': 12,
 'burn': 13,
 'by': 14,
 'caught': 15,
 'close': 16,
 'could': 17,
 'crying': 18,
 'curse': 19,
 'danced': 20,
 'dark': 21,
 'day': 22,
 'death': 23,
 'deeds': 24,
 'do': 25,
 'dying': 26,
 'end': 27,
 'eyes': 28,
 'father': 29,
 'fierce': 30,
 'flight': 31,
 'forked': 32,
 'frail': 33,
 'gay': 34,
 'gentle': 35,
 'go': 36,
 'good': 37,
 'grave': 38,
 'green': 39,
 'grieved': 40,
 'had': 41,
 'have': 42,
 'height': 43,
 'how': 44,
 'i': 45,
 'in': 46,
 'into': 47,
 'is': 48,
 'it': 49,
 'its': 50,
 'know': 51,
 'last': 52,
 'late': 53,
 'learn': 54,
 'light': 55,
 'lightning': 56,
 'like': 57,
 'me': 58,
 'men': 59,
 'meteors': 60,
 'might': 61,
 'my': 62,
 'near': 63,
 'night': 64,
 'no': 65,
 'not': 66,
 'now': 67,
 'of': 68,
 'old': 69,
 'on': 70,
 'pray': 71,
 'rage': 72,
 'rave': 73,
 'right': 74,
 'sad': 75,
 'sang': 7

In [43]:
print vec_mat.shape
print vec_mat

(19, 98)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]]


In [44]:
print lines[2]
print vec_mat[2]

Rage, rage against the dying of the light.
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.]


In [45]:
sk_vectorizer = CountVectorizer()
sk_vec_mat = sk_vectorizer.fit_transform(lines).toarray()

In [46]:
sk_vectorizer.get_feature_names()

[u'against',
 u'age',
 u'and',
 u'at',
 u'bay',
 u'be',
 u'because',
 u'blaze',
 u'bless',
 u'blind',
 u'blinding',
 u'bright',
 u'burn',
 u'by',
 u'caught',
 u'close',
 u'could',
 u'crying',
 u'curse',
 u'danced',
 u'dark',
 u'day',
 u'death',
 u'deeds',
 u'do',
 u'dying',
 u'end',
 u'eyes',
 u'father',
 u'fierce',
 u'flight',
 u'forked',
 u'frail',
 u'gay',
 u'gentle',
 u'go',
 u'good',
 u'grave',
 u'green',
 u'grieved',
 u'had',
 u'have',
 u'height',
 u'how',
 u'in',
 u'into',
 u'is',
 u'it',
 u'its',
 u'know',
 u'last',
 u'late',
 u'learn',
 u'light',
 u'lightning',
 u'like',
 u'me',
 u'men',
 u'meteors',
 u'might',
 u'my',
 u'near',
 u'night',
 u'no',
 u'not',
 u'now',
 u'of',
 u'old',
 u'on',
 u'pray',
 u'rage',
 u'rave',
 u'right',
 u'sad',
 u'sang',
 u'see',
 u'should',
 u'sight',
 u'sun',
 u'tears',
 u'that',
 u'the',
 u'their',
 u'there',
 u'they',
 u'though',
 u'too',
 u'wave',
 u'way',
 u'who',
 u'wild',
 u'wise',
 u'with',
 u'words',
 u'you',
 u'your']

In [47]:
sk_vectorizer.vocabulary_

{u'against': 0,
 u'age': 1,
 u'and': 2,
 u'at': 3,
 u'bay': 4,
 u'be': 5,
 u'because': 6,
 u'blaze': 7,
 u'bless': 8,
 u'blind': 9,
 u'blinding': 10,
 u'bright': 11,
 u'burn': 12,
 u'by': 13,
 u'caught': 14,
 u'close': 15,
 u'could': 16,
 u'crying': 17,
 u'curse': 18,
 u'danced': 19,
 u'dark': 20,
 u'day': 21,
 u'death': 22,
 u'deeds': 23,
 u'do': 24,
 u'dying': 25,
 u'end': 26,
 u'eyes': 27,
 u'father': 28,
 u'fierce': 29,
 u'flight': 30,
 u'forked': 31,
 u'frail': 32,
 u'gay': 33,
 u'gentle': 34,
 u'go': 35,
 u'good': 36,
 u'grave': 37,
 u'green': 38,
 u'grieved': 39,
 u'had': 40,
 u'have': 41,
 u'height': 42,
 u'how': 43,
 u'in': 44,
 u'into': 45,
 u'is': 46,
 u'it': 47,
 u'its': 48,
 u'know': 49,
 u'last': 50,
 u'late': 51,
 u'learn': 52,
 u'light': 53,
 u'lightning': 54,
 u'like': 55,
 u'me': 56,
 u'men': 57,
 u'meteors': 58,
 u'might': 59,
 u'my': 60,
 u'near': 61,
 u'night': 62,
 u'no': 63,
 u'not': 64,
 u'now': 65,
 u'of': 66,
 u'old': 67,
 u'on': 68,
 u'pray': 69,
 u'rage': 70

In [48]:
sk_vec_mat[2]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])

In [49]:
pairwise_dist = squareform(pdist(vec_mat, metric='cosine'))
pairwise_dist.shape

(19, 19)

In [50]:
for i in xrange(len(lines[:5])):
    for ind in pairwise_dist[i].argsort()[:5]:
        print lines[ind]
    print '-----'

Do not go gentle into that good night,
Do not go gentle into that good night.
Do not go gentle into that good night.
Do not go gentle into that good night.
Good men, the last wave by, crying how bright
-----
Old age should burn and rave at close of day;
And you, my father, there on the sad height,
Blind eyes could blaze like meteors and be gay,
Wild men who caught and sang the sun in flight,
Though wise men at their end know dark is right,
-----
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
And you, my father, there on the sad height,
-----
Though wise men at their end know dark is right,
Because their words had forked no lightning they
Good men, the last wave by, crying how bright
Grave men, near death, who see with blinding sight
Wild men who caught and sang the sun in flight,
-----
Because their words had forked no lightning they
Though wise men at their end 

In [51]:
tf_vec_mat, tf_voc_list, tf_voc_dict = my_vectorizer(lines, kind='tfidf')

In [52]:
tf_vec_mat[2]

array([ 0.        ,  0.1732868 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.1732868 ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.1732868 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.13732654,  0.  

In [53]:
sk_tf_vec = TfidfVectorizer()
sk_tf_mat = sk_tf_vec.fit_transform(lines).toarray()

In [54]:
sk_tf_mat[2]

array([ 0.30960308,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.30960308,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.30960308,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.28594828,  0.        ,  0.        ,  0.  

In [55]:
sum(sk_tf_mat[2]**2)

1.0000000000000002

In [56]:
pairwise_dist_tf = squareform(pdist(tf_vec_mat, metric='cosine'))
pairwise_dist_tf.shape

(19, 19)

In [57]:
# tfidf vector similarity
for i in xrange(len(lines[:5])):
    for ind in pairwise_dist_tf[i].argsort()[:5]:
        print lines[ind]
    print '-----'

Do not go gentle into that good night,
Do not go gentle into that good night.
Do not go gentle into that good night.
Do not go gentle into that good night.
Good men, the last wave by, crying how bright
-----
Old age should burn and rave at close of day;
Though wise men at their end know dark is right,
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
-----
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Old age should burn and rave at close of day;
-----
Though wise men at their end know dark is right,
Old age should burn and rave at close of day;
Because their words had forked no lightning they
Their frail deeds might have danced in a green bay,
Good men, the last wave by, crying how bright
-----
Because their words had forked no lightning they
And learn, too late, they grieved it on i

In [58]:
# count vector similarity
for i in xrange(len(lines[:5])):
    for ind in pairwise_dist[i].argsort()[:5]:
        print lines[ind]
    print '-----'

Do not go gentle into that good night,
Do not go gentle into that good night.
Do not go gentle into that good night.
Do not go gentle into that good night.
Good men, the last wave by, crying how bright
-----
Old age should burn and rave at close of day;
And you, my father, there on the sad height,
Blind eyes could blaze like meteors and be gay,
Wild men who caught and sang the sun in flight,
Though wise men at their end know dark is right,
-----
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
Rage, rage against the dying of the light.
And you, my father, there on the sad height,
-----
Though wise men at their end know dark is right,
Because their words had forked no lightning they
Good men, the last wave by, crying how bright
Grave men, near death, who see with blinding sight
Wild men who caught and sang the sun in flight,
-----
Because their words had forked no lightning they
Though wise men at their end 